# Zadania - metody optymalizacji

### 7.1 AdaGrad

1. Rozszerz poniższą implementację algorytmu SGD o funkcjonalność algorytmu AdaGrad (włączaną parametrem `adaGrad=True`). Zasady działania AdaGrad są podane w materiałach do wykładu.
   <br />**Uwaga**: podczas dzielenia przez pierwastek sumy kwadratów historycznych gradientów warto dodać małą wartość $\epsilon=10^{-7}$ do mianownika, aby uniknąć dzielenia przez 0.
   
1. Stwórz model wieloklasowej regresji logistycznej na zbiorze MNIST (dla wszystkich 10 cyfr) i oblicz jego jakość na zbiorze testowym. Zastosuj poniższe parametry:
 1. Rozmiar wsadu: 50
 1. Liczba epok: 2
 1. Rozmiar kroku $\alpha$: 1.0 (w algorytmie AdaGrad $\alpha$ może być niezmienne)
1. Spróbuj uzyskać wynik podobny lub lepszy samym algorytmem SGD bez opcji AdaGrad, dostrajając parametry $\alpha$. Czy jest to możliwe?

### 7.2 Ensemble

1. Na danych MNIST wytrenuj 10 klasyfikatorów z sensownie dobranymi parametrami dla algorytmu SGD. Zastosuj randomizację zbioru uczącego dla każdego modelu i w każdej epoce.<br />
**Uwaga**: pamiętaj, aby tasować $X$ i $Y$ w tej samej kolejności! 
1. Oblicz jakość każdego z klasyfikatorów na zbiorze testowym.
1. Oblicz jakość predykcji (klas) uzyskanych w wyniku wybranej metody agregacji wyników: głosowania klas lub uśredniania prawdopodobieństw, i porównaj z wcześniej uzyskanymi wynikami. Jak wynik z metody zbiorczej odnosi się do wyników uzyskiwanych przez pojedyncze klasyfikatory?

In [1]:
import os
import struct
import numpy as np
import itertools
from statistics import mode

%matplotlib inline

def readMnist(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def showImage(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

In [2]:
def mnistMatrix(data, maxItems=1000):
    datalist = [t for t in data]
    m = maxItems if maxItems != -1 else len(datalist)
    n = 28 * 28 + 1
    X = np.matrix(np.zeros(m * n)).reshape(m, n)
    Y = np.matrix(np.zeros(m)).reshape(m, 1)
    for i, (label, image) in enumerate(datalist[:m]):
        X[i, 0] = 1 # bias term
        X[i, 1:] = image.reshape(28*28,)
        Y[i] = label
    return X, Y

## AdaGrad

In [3]:
X, y = mnistMatrix(readMnist(dataset='training'), maxItems=-1)
X_test, y_test = mnistMatrix(readMnist(dataset='testing'), maxItems=-1)

In [4]:
def normalize(X):
    X[:,1:] = X[:,1:] / 255
    return X

def mapY(y, cls):
    m = len(y)
    yBi = np.matrix(np.zeros(m)).reshape(m, 1)
    yBi[y == cls] = 1.
    return yBi

def indicatorMatrix(y):
    classes = np.unique(y.tolist())
    m = len(y)
    k = len(classes)
    Y = np.matrix(np.zeros((m, k)))
    for i, cls in enumerate(classes):
        Y[:,i] = mapY(y, cls)
    return Y

def safeSigmoid(x, eps=0):
    y = 1.0/(1.0 + np.exp(-x))
    if eps > 0:
        y[y < eps] = eps
        y[y > 1 - eps] = 1 - eps
    return y

def h(theta, X, eps=0.0):
    return safeSigmoid(X*theta, eps)

def J(h,theta,X,y):
    m = len(y)
    f = h(theta, X, eps=10**-7)
    return -np.sum(np.multiply(y, np.log(f)) + 
                   np.multiply(1 - y, np.log(1 - f)), axis=0)/m

def dJ(h,theta,X,y):
    return 1.0/len(y)*(X.T*(h(theta,X)-y))

def softmax(X):
    return np.exp(X)/np.sum(np.exp(X))

In [5]:
def SGD(h, fJ, fdJ, theta, X, y, 
        alpha=0.001, maxEpochs=1, batchSize=100, adaGrad=True, shuffle=False):
    m, n = X.shape
    start, end = 0, batchSize
    eps = 10 ** -7
    historic_grads = np.matrix(np.zeros(n)).reshape(n, 1)
    maxSteps = (m * float(maxEpochs)) / batchSize
   
    count = 0
    for i in range(int(maxSteps)):
        if shuffle and (i * batchSize) % m == 0:
            indexes = [j for j in range(len(X))]
            np.random.shuffle(indexes)
            X = X[indexes]
            y = y[indexes]
        
        XBatch, yBatch =  X[start:end,:], y[start:end,:]
        grad = fdJ(h, theta, XBatch, yBatch)

        if adaGrad:
            historic_grads += np.multiply(grad, grad)
            adjusted_grad = np.multiply(alpha / (np.sqrt(historic_grads) + eps), grad)
        else:
            adjusted_grad = alpha * grad

        theta = theta - adjusted_grad
        
        if start + batchSize < m:
            start += batchSize
        else:
            start = 0
        end = min(start + batchSize, m)
        
    return theta

def classify(thetas, X, debug=False):
    regs = np.array([(X*theta).item() for theta in thetas])
    if debug:
        print("regs  =", regs)
    probs = softmax(regs)
    if debug:
        print("probs =", np.around(probs, decimals=3))
    return np.argmax(probs), probs

def trainMaxEnt(X, Y, alpha = 1.0, maxEpochs = 2, batchSize = 50, adaGrad = True, shuffle = False):
    n = X.shape[1]
    thetas = []
    for c in range(Y.shape[1]):
        YBi = Y[:,c]
        theta = np.matrix(np.random.random(n)).reshape(n,1)
        thetaBest = SGD(h, J, dJ, theta, X, YBi, alpha, maxEpochs, batchSize, adaGrad, shuffle)
        thetas.append(thetaBest)
    return thetas

def calculateAcc(thetas, X_test, y_test, debug=True):
    acc = 0.0
    for i in range(len(y_test)):
        cls, probs = classify(thetas, X_test[i])
        correctCls = int(y_test[i].item())
        if i < 6 and debug:
            print(correctCls, "  <=>", cls, " -- ", cls == correctCls, np.round(probs, 4).tolist())
        acc += correctCls == cls
    return acc

def calculateAccReturnClasses(thetas, X_test, y_test, debug=True):
    acc = 0.0
    predictions = []
    for i in range(len(y_test)):
        cls, probs = classify(thetas, X_test[i])
        correctCls = int(y_test[i].item())
        if i < 6 and debug:
            print(correctCls, "  <=>", cls, " -- ", cls == correctCls, np.round(probs, 4).tolist())
        acc += correctCls == cls
        predictions.append(cls)
    return acc, predictions

In [6]:
X_norm = normalize(X)
X_test_norm = normalize(X_test)
ind_y = indicatorMatrix(y)

In [7]:
gd_ada = trainMaxEnt(X_norm, ind_y, alpha = 1.0, maxEpochs = 2, batchSize = 50, adaGrad = True)
print("Accuracy with Ada = %f\n" % (calculateAcc(gd_ada, X_test_norm, y_test)/len(X_test_norm)))

gd_without_ada = trainMaxEnt(X_norm, ind_y, alpha = 0.1, maxEpochs = 2, batchSize = 50, adaGrad = False)
print("Accuracy without Ada = %f\n" % (calculateAcc(gd_without_ada, X_test_norm, y_test)/len(X_test_norm)))

7   <=> 7  --  True [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
2   <=> 2  --  True [0.0, 0.0, 0.9875, 0.0, 0.0, 0.0006, 0.0119, 0.0, 0.0, 0.0]
1   <=> 1  --  True [0.0, 0.9999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0   <=> 0  --  True [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
4   <=> 4  --  True [0.0, 0.0, 0.0001, 0.0, 0.9983, 0.0, 0.0001, 0.0, 0.0, 0.0015]
1   <=> 1  --  True [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Accuracy with Ada = 0.890600

7   <=> 7  --  True [0.0, 0.0, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.9999, 0.0, 0.0]
2   <=> 2  --  True [0.0012, 0.0, 0.9745, 0.0003, 0.0, 0.0064, 0.017, 0.0, 0.0005, 0.0]
1   <=> 1  --  True [0.0, 0.9981, 0.0002, 0.0003, 0.0, 0.0004, 0.0003, 0.0003, 0.0003, 0.0002]
0   <=> 0  --  True [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
4   <=> 4  --  True [0.0, 0.0, 0.0002, 0.0, 0.99, 0.0, 0.0003, 0.0007, 0.0006, 0.0082]
1   <=> 1  --  True [0.0, 0.9978, 0.0001, 0.0005, 0.0, 0.0, 0.0, 0.0006, 0.0006, 0.0002]
Accuracy with

Udało się uzyskać lepszy wynik, jednak wymagało to wypróbowania kilku możliwości parametru alpha i czasu na przetrenowanie danych po raz kolejny.

## Ensemble

In [8]:
def most_common(lst):
    return max(set(lst), key=lst.count)

models_predictions = []
for i in range(10):
    indexes = [j for j in range(len(X_norm))]
    np.random.shuffle(indexes)
    X_norm = X_norm[indexes]
    ind_y = ind_y[indexes]
    
    gd = trainMaxEnt(X_norm, ind_y, alpha=0.1, maxEpochs=3, batchSize=50, adaGrad=True, shuffle=True)
    acc, predicts = calculateAccReturnClasses(gd, X_test_norm, y_test, debug=False)
    models_predictions.append(predicts)
    print("Accuracy model %d = %f" % (i, acc/len(X_test_norm)))
    
acc = 0.0
for i in range(len(y_test)):
    predictions_row = []
    for j in range(len(models_predictions)):
        predictions_row.append(models_predictions[j][i])
    freq = most_common(predictions_row)
    correctCls = int(y_test[i].item())
    acc += correctCls == freq
    
print("\nTotal accuracy = %f" % (acc/len(X_test_norm)))
    

Accuracy model 0 = 0.910800
Accuracy model 1 = 0.907600
Accuracy model 2 = 0.907300
Accuracy model 3 = 0.907600
Accuracy model 4 = 0.910000
Accuracy model 5 = 0.909300
Accuracy model 6 = 0.908000
Accuracy model 7 = 0.907600
Accuracy model 8 = 0.908500
Accuracy model 9 = 0.909700

Total accuracy = 0.911100


Ostatecznie udało się uzyskać accuracy wyższe niż w przypadku każdego z pojedynczych modeli, użycie Ensemble polepszyło wynik.